# 第二章 模型，提示和输出解释器

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、直接使用OpenAI](#二、直接使用OpenAI)
     - [2.1 计算1+1](#2.1-计算1+1)
     - [2.2 用美式英语表达海盗邮件](#2.2-用美式英语表达海盗邮件)
     - [2.3 中文版](#2.3-中文版)
 - [三、通过LangChain使用OpenAI](#三、通过LangChain使用OpenAI)
     - [3.1 模型](#3.1-模型)
     - [3.2 提示模板](#3.2-提示模板)
         - [3.2.1 使用LangChain提示模版](#3.2.1-使用LangChain提示模版)
         - [3.2.2 中文版](#3.2.2-中文版)
         - [3.2.2 为什么需要提示模版](#3.2.2-为什么需要提示模版)
     - [3.3 输出解析器](#3.3-输出解析器)
         - [3.3.1 如果没有输出解析器](#3.3.1-如果没有输出解析器)
         - [3.3.2 中文版](#3.3.2-中文版)
         - [3.3.3 LangChain输出解析器](#3.3.3-LangChain输出解析器)
         - [3.3.4 中文版](#3.3.4-中文版)
 - [四、补充材料](#四、补充材料)
     - [4.1 链式思考推理(ReAct)](#4.1-链式思考推理(ReAct))


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

## 二、直接使用OpenAI

我们先从直接调用OpenAI的API开始。

`get_completion`函数是基于`openai`的封装函数，对于给定提示（prompt）输出相应的回答。其包含两个参数
   
   - `prompt` 必需输入参数。 你给模型的**提示，可以是一个问题，可以是你需要模型帮助你做的事**（改变文本写作风格，翻译，回复消息等等）。
   - `model` 非必需输入参数。默认使用gpt-3.5-turbo。你也可以选择其他模型。
   
这里的提示对应我们给chatgpt的问题，函数给出的输出则对应chatpgt给我们的答案。

In [2]:
def get_completion(prompt, model="gpt-4o-mini"):
    
    messages = [{"role": "user", "content": prompt}]
    client = OpenAI(api_key=openai_api_key)

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

### 2.1 计算1+1

我们来一个简单的例子 - 分别用中英文问问模型

- 中文提示(Prompt in Chinese)： `1+1是什么？`
- 英文提示(Prompt in English)： `What is 1+1?`

In [3]:
# 中文
get_completion("1+1是什么？")

'1+1等于2。'

In [4]:
# 英文
get_completion("What is 1+1?")

'1 + 1 equals 2.'

### 2.2 用美式英语表达海盗邮件

上面的简单例子，模型`gpt-3.5-turbo`<span style="color:#00ff00">（我们用的是gpt-4o-mini）</span>对我们的关于1+1是什么的提问给出了回答。

现在我们来看一个复杂一点的例子： 

假设我们是电商公司员工，我们的顾客是一名海盗A，他在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是海盗A给我们的客服中心写来以下邮件：`customer_email`

In [17]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

我们的客服人员对于海盗的措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用美式英语的表达方式将海盗的邮件进行翻译，客服人员可以更好理解。*这里海盗的英文表达可以理解为英文的方言，其与美式英语的关系，就如四川话与普通话的关系。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

根据这两个小目标，定义一下文本表达风格：`style`

In [18]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""

下一步需要做的是将`customer_email`和`style`结合起来构造我们的提示:`prompt`

In [19]:
# 要求模型根据给出的语调进行转化
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



`prompt` 构造好了，我们可以调用`get_completion`得到我们想要的结果 - 用平和尊重的语气，美式英语表达的海盗语言邮件

In [20]:
response = get_completion(prompt)

In [21]:
response

'I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would appreciate your assistance with this issue. Thank you.'

对比语言风格转换前后，用词更为正式，替换了极端情绪的表达，并表达了感谢。

✨ 你可以尝试修改提示，看可以得到什么不一样的结果😉

### 2.3 中文版

In [13]:
# 普通话 + 平静、尊敬的语调
style = """正式普通话 \
用一个平静、尊敬的语调
"""

In [14]:
# 非正式用语
customer_email = """   
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

In [15]:
# 要求模型根据给出的语调进行转化
prompt = f"""把由三个反引号分隔的文本\
翻译成{style}。
文本: ```{customer_email}```
"""

print(prompt)

把由三个反引号分隔的文本翻译成正式普通话 用一个平静、尊敬的语调
。
文本: ```   
阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！
```



In [16]:
response = get_completion(prompt)

response

'我感到非常沮丧，因为我的搅拌机盖子掉落，导致奶昔溅到了厨房的墙壁上。更糟糕的是，保修并不包括清理厨房的费用。我现在需要您的帮助。'

## 三、通过LangChain使用OpenAI

在前面一部分，我们通过封装函数`get_completion`直接调用了OpenAI完成了对方言邮件进行了的翻译，得到用平和尊重的语气、正式的普通话表达的邮件。

让我们尝试使用LangChain来实现相同的功能。

### 3.1 模型

从`langchain_community.chat_models.openai`导入`OpenAI`的对话模型`ChatOpenAI`。 除去OpenAI以外，`langchain_community.chat_models`还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)。

<span style="color:#00ff00">
参考Langchain官方的最新文档，我们从`langchain.chat_models`导入`init_chat_model`函数，通过在`init_chat_model`函数中指定模型名称和模型提供商，来初始化模型。
</span>

In [31]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini",model_provider="openai", temperature=0)
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1118589d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x111850a90>, root_client=<openai.OpenAI object at 0x111831cd0>, root_async_client=<openai.AsyncOpenAI object at 0x11185bed0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

上面的输出显示ChatOpenAI的默认模型为`gpt-3.5-turbo`（<span style="color:#00ff00">可以看到，我们此时的模型为gpt-4o-mini</span>）

### 3.2 提示模板

在前面的例子中，我们通过[f字符串](https://docs.python.org/zh-cn/3/tutorial/inputoutput.html#tut-f-strings)把Python表达式的值`style`和`customer_email`添加到`prompt`字符串内。

```python
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""
```
`langchain`提供了接口方便快速的构造和使用提示。现在我们来看看如何使用`langchain`来构造提示。

#### 3.2.1 使用LangChain提示模版

##### 1️⃣ 构造提示模版字符串
我们构造一个提示模版字符串：`template_string`

In [52]:
template_string = """把由三个反引号分隔的文本翻译成{style}。\
文本: ```{text}```
"""

##### 2️⃣ 构造LangChain提示模版
我们调用`ChatPromptTemplate.from_template()`函数将上面的提示模版字符`template_string`转换为提示模版`prompt_template`

In [53]:
from langchain.prompts.chat import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [54]:
print(prompt_template.messages[0].prompt)

input_variables=['style', 'text'] input_types={} partial_variables={} template='把由三个反引号分隔的文本翻译成{style}。文本: ```{text}```\n'


从上面的输出可以看出，`prompt_template` 有两个输入变量： `style` 和 `text`。

In [55]:
print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


##### 3️⃣ 使用模版得到客户消息提示

langchain提示模版`prompt_template`需要两个输入变量： `style` 和 `text`。 这里分别对应 
- `customer_style`: 我们想要的顾客邮件风格
- `customer_email`: 顾客的原始邮件文本。

In [56]:
customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

对于给定的`customer_style`和`customer_email`, 我们可以使用提示模版`prompt_template`的`format_messages`方法生成想要的客户消息`customer_messages`。

In [57]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [58]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


可以看出`customer_messages`变量类型为列表(`list`)，而列表里的元素变量类型为langchain自定义消息(`langchain.schema.HumanMessage`)。<span style="color:#00ff00">现在是langchain_core.messages.human.HumanMessage类型</span>

打印第一个元素可以得到如下:

In [59]:
print(customer_messages[0])

content='把由三个反引号分隔的文本翻译成正式普通话 用一个平静、尊敬的语气\n。文本: ```\n阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！\n```\n' additional_kwargs={} response_metadata={}


##### 4️⃣ 调用chat模型转换客户消息风格

现在我们可以调用[模型](#model)部分定义的chat模型来实现转换客户消息风格。到目前为止，我们已经实现了在前一部分的任务。

In [61]:
customer_response = model.invoke(customer_messages, temperature=0.0)

<span style="color:#00ff00">可以如下流式输出：</span>

In [68]:
for token in model.stream(customer_messages, temperature=0.7):
    print(token.content, end="|")

|我|感|到|非常|沮|丧|，因为|我的|搅|拌|机|盖|子|掉|落|，|导致|奶|昔|溅|到了|厨房|的|墙|上|。|更|令人|头|痛|的是|，|保|修|并|不|包括|清|理|厨房|的|费用|。我|现在|需要|您的|帮助|。||

In [62]:
print(customer_response.content)

我感到非常沮丧，因为我的搅拌机盖子掉落，导致奶昔溅到了厨房的墙壁上。更糟糕的是，保修并不包括清理厨房的费用。我现在需要您的帮助。


##### 5️⃣ 使用模版得到回复消息提示

接下来，我们更进一步，将客服人员回复的消息，转换为海盗的语言风格，并确保消息比较有礼貌。 

这里，我们可以继续使用第2️⃣步构造的langchain提示模版，来获得我们回复消息提示。

In [63]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""

In [64]:
service_style_pirate = """\
一个有礼貌的语气 \
使用正式的普通话 \
"""

In [65]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

把由三个反引号分隔的文本翻译成一个有礼貌的语气 使用正式的普通话 。文本: ```嘿，顾客， 保修不包括厨房的清洁费用， 因为您在启动搅拌机之前 忘记盖上盖子而误用搅拌机, 这是您的错。 倒霉！ 再见！
```



##### 6️⃣ 调用chat模型转换回复消息风格

调用[模型](#model)部分定义的model模型来转换回复消息风格

In [66]:
service_response = model.invoke(service_messages)
print(service_response.content)

尊敬的顾客：

您好！感谢您与我们联系。关于您的问题，我想向您说明，保修条款中并不包括厨房清洁费用。由于在使用搅拌机之前未盖上盖子而导致的误用，责任在于您本人。对此我们深感遗憾。

感谢您的理解，祝您一切顺利！

此致  
敬礼


#### 3.2.2 为什么需要提示模版

在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。**使用提示模版，可以让我们更为方便地<span style="color:red">重复使用设计好的提示</span>**。

下面给出了一个比较长的提示模版案例。学生们线上学习并提交作业，通过以下的提示来实现对学生的提交的作业的评分。

In [36]:
# 中文版
prompt = """ 你的任务是判断学生的解决方案是正确的还是不正确的

要解决该问题，请执行以下操作：
 - 首先，制定自己的问题解决方案
 - 然后将您的解决方案与学生的解决方案进行比较
 并评估学生的解决方案是否正确。
...
使用下面的格式进行回答:

问题:
```
问题文本
```
学生的解决方案:
```
学生的解决方案文本
```
实际解决方案:
```
...
制定解决方案的步骤以及您的解决方案请参见此处
```
学生的解决方案和实际解决方案是否相同 \
只计算：
```
是或者不是
```
学生的成绩
```
正确或者不正确
```

问题:
```
{question}
```
学生的解决方案:
```
{student's solution}
```
实际解决方案:

"""

此外，LangChain还提供了提示模版用于一些常用场景。比如自动摘要、问答、连接到SQL数据库、连接到不同的API. 通过使用LangChain<span style="color:red">内置</span>的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

最后，我们在建立大模型应用时，通常希望模型的输出为给定的格式，比如在输出使用特定的关键词来让<span style="color:red">输出结构化</span>。 下面为一个使用大模型进行链式思考推理例子，对于问题：*What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?*

通过使用LangChain库函数，输出采用"Thought"（思考）、"Action"（行动）、"Observation"（观察）作为<span style="color:red">链式思考推理</span>的关键词，让输出结构化。

```
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]
```

在补充材料中，可以查看使用LangChain和OpenAI进行链式思考推理的另一个代码实例。

### 3.3 输出解析器

#### 3.3.1 如果没有输出解析器

对于给定的评价`customer_review`, 我们希望提取信息，并按以下格式输出：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [89]:
customer_review = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

##### 1️⃣ 构造提示模版字符串

In [97]:
review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

请注意，输出中不要包含```json字样或任何其他字符。

文本: {text}
"""

##### 2️⃣ 构造langchain提示模版

In [98]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='对于以下文本，请从中提取以下信息：\n\n礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。\n\n交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。\n\n价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。\n\n使用以下键将输出格式化为 JSON：\n礼物\n交货天数\n价钱\n\n请注意，输出中不要包含```json字样或任何其他字符。\n\n文本: {text}\n'), additional_kwargs={})]


##### 3️⃣ 使用模版得到提示消息

In [99]:
messages = prompt_template.format_messages(text=customer_review)
print(messages[0].content)

对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

请注意，输出中不要包含```json字样或任何其他字符。

文本: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。




##### 4️⃣ 调用chat模型提取信息

In [100]:
model = init_chat_model("gpt-4o-mini",model_provider="openai", temperature=0)
response = model.invoke(messages)
print(response.content)

{
  "礼物": "是的",
  "交货天数": 2,
  "价钱": ["它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的"]
}


##### 📝 分析与总结
`response.content`类型为字符串（`str`），而并非字典(`dict`), 直接使用`get`方法会报错。因此，我们需要输出解释器。

In [101]:
type(response.content)

str

In [102]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

#### 3.3.3 LangChain输出解析器

##### 1️⃣ 构造提示模版字符串

In [103]:
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

##### 2️⃣ 构造langchain提示模版

In [104]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

##### 3️⃣ 构造输出解析器

<span style="color:#00ff00">
这里的标准输出解析格式是Json格式，键由ResponseSchema的name属性决定。description属性则定义了键的描述，作为让llm进行输出的prompt的一部分。
</span>

In [105]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？\
                            如果是，则回答 是的，\
                            如果否或未知，则回答 不是。")

delivery_days_schema = ResponseSchema(name="交货天数",
                                      description="产品需要多少天才能到达？\
                                      如果没有找到该信息，则输出-1。")

price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，\
                                    并将它们输出为逗号分隔的 Python 列表")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```


##### 3️⃣ 使用模版得到提示消息

<span style="color:#00ff00">
通过format_instructions，我们可以将预定义的输出格式添加到提示模版中，便于后续的输出解析。
</span>

In [106]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [107]:
print(messages[0].content)

对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```



##### 4️⃣ 调用模型提取信息

In [108]:
response = model.invoke(messages)
print(response.content)

```json
{
	"礼物": "是的",
	"交货天数": "2",
	"价钱": "它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的"
}
```


##### 5️⃣ 使用输出解析器解析输出

In [109]:
output_dict = output_parser.parse(response.content)
output_dict

{'礼物': '是的', '交货天数': '2', '价钱': '它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的'}

<span style="color:#00ff00">
parse函数本质是解析输入中的json字符串，并转换为字典。注意，输入的json字符串需要使用```json包裹。此外，该函数要求json字符串的键由output_parser.response_schemas中定义的ResponseSchema的name属性决定。
</span>

In [115]:
output_parser.response_schemas

[ResponseSchema(name='礼物', description='这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。', type='string'),
 ResponseSchema(name='交货天数', description='产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。', type='string'),
 ResponseSchema(name='价钱', description='提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表', type='string')]

In [114]:
example = """
```json
{
  "礼物": true,
  "交货天数": 2,
  "价钱": "它比其他吹叶机稍微贵一点"
}
```
"""
output_dict = output_parser.parse(example)
output_dict

{'礼物': True, '交货天数': 2, '价钱': '它比其他吹叶机稍微贵一点'}

##### 📝 分析与总结
`output_dict`类型为字典(`dict`), 可直接使用`get`方法。这样的输出更方便下游任务的处理。

In [110]:
type(output_dict)

dict

In [112]:
delivery_day = output_dict.get('交货天数')
delivery_day

'2'

## 四、补充材料

### 4.1 链式思考推理(ReAct)
参考资料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

<span style="color:#00ff00">
由于搜索工具是基于维基百科的，所以需要使用英文进行搜索。使用中文问题的效果不好。以下仅仅是一个简单的ReAct示例。
</span>

<span style="color:#00ff00">
该实现采用的是旧版的api，新版api的实现可参考第7章。
</span>

In [88]:
from langchain_community.docstore.wikipedia import Wikipedia
from langchain.chat_models import init_chat_model
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = init_chat_model("gpt-4o-mini",model_provider="openai", temperature=0)

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)

# 运行示例问题
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
result = agent_executor.run(question)
print("\nFinal Answer:", result) 



> Entering new AgentExecutor chain...
Thought: I need to search for David Chanoff and find out which U.S. Navy admiral he collaborated with, then determine under which President that admiral served as ambassador to the United Kingdom.  
Action: Search[David Chanoff]  
Observation: David Chanoff (born November 15, 1943, in Philadelphia) is an American author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman, among others. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine.
Chanoff founded Sudbury Valley School in Framingham, Massachusetts in 1968. In the late 1970s and early 1980s, he taught English at Tufts University and Harvard University. 
In 20